# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", # Longitude for the x-axis
    "Lat", # Latitude for the y-axis
    geo = True, # Enable geographic plotting
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:

# Narrow down cities that fit criteria and drop any results with null values
# Define the conditions for the cities you want to filter (example: temp between 21°C and 27°C and wind speed greater than 5)

min_temp_s = city_data_df["Max Temp"] > 21     # Create a boolean series where each entry is True if the city's Max Temp is greater than 21°C, and False otherwise

max_temp_s = city_data_df["Max Temp"] < 27     # Create a boolean series where each entry is True if the city's Max Temp is less than 27°C, and False otherwise

wind_speed_s = city_data_df["Wind Speed"] > 5  # Create a boolean series where each entry is True if the city's Wind Speed is greater than 5, and False otherwise

# Combine the conditions using the & operator (logical AND)
# This will give True for rows where all conditions are met, and False otherwise
ideal_cities_s = min_temp_s & max_temp_s & wind_speed_s

# Apply the conditions to filter the DataFrame
filtered_cities_df = city_data_df[ideal_cities_s]

# Drop any rows that have null values
filtered_cities_df = filtered_cities_df.dropna()

# Display the filtered data with the correct columns and order as in the screenshot
filtered_cities_df = filtered_cities_df[['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]

# Display the filtered DataFrame (the cities that meet the criteria and have no missing values)
filtered_cities_df.head(20)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
34,34,naberera,-4.2000,36.9333,21.12,54,94,6.38,TZ,1666108252
46,46,mogadishu,2.0371,45.3438,25.67,83,13,6.45,SO,1666108258
64,64,atuona,-9.8000,-139.0333,24.69,69,1,7.05,PF,1666108268
65,65,hermanus,-34.4187,19.2345,21.56,80,9,6.07,ZA,1666108269
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
93,93,ugoofaaru,5.6667,73.0000,26.95,80,100,8.28,MV,1666108290
99,99,trincomalee,8.5711,81.2335,26.66,82,100,6.96,LK,1666108294
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219
107,107,inhambane,-23.8650,35.3833,26.99,65,40,5.81,MZ,1666108299


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a new DataFrame called hotel_df
# Copy the relevant columns from the filtered DataFrame
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column "Hotel Name" to store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display the sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,None
34,naberera,TZ,-4.2000,36.9333,54,None
46,mogadishu,SO,2.0371,45.3438,83,None
64,atuona,PF,-9.8000,-139.0333,69,None
65,hermanus,ZA,-34.4187,19.2345,80,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories": "accommodation.hotel",
    "limit": 1,  # We only want the nearest hotel
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
naberera - nearest hotel: No hotel found
mogadishu - nearest hotel: Villa Rays Restaurant & Hotel
atuona - nearest hotel: Pearl Resort
hermanus - nearest hotel: Aloe guest house
nalut - nearest hotel: No hotel found
ugoofaaru - nearest hotel: No hotel found
trincomalee - nearest hotel: Summer set inn
palencia - nearest hotel: Hotel Área de Servicio Los Chopos
inhambane - nearest hotel: Hotel de Inhambane
faanui - nearest hotel: Oa Oa Lodge
ocos - nearest hotel: No hotel found
peniche - nearest hotel: Residencial Maciel
maragogi - nearest hotel: Pousada Mariluz
moroni - nearest hotel: Ville de Moroni
rawson - nearest hotel: Hotel Deportivo
quatre cocos - nearest hotel: Horisun
comodoro rivadavia - nearest hotel: No hotel found
camacha - nearest hotel: Praia Dourada
kloulklubed - nearest hotel: The Adventures Inn
eyl - nearest hotel: فندق ايل دابي
tandil - nearest hotel: Viñas del Rosario Hotel Boutique
vao - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
34,naberera,TZ,-4.2000,36.9333,54,No hotel found
46,mogadishu,SO,2.0371,45.3438,83,Villa Rays Restaurant & Hotel
64,atuona,PF,-9.8000,-139.0333,69,Pearl Resort
65,hermanus,ZA,-34.4187,19.2345,80,Aloe guest house
73,nalut,LY,30.3333,10.8500,27,No hotel found
93,ugoofaaru,MV,5.6667,73.0000,80,No hotel found
99,trincomalee,LK,8.5711,81.2335,82,Summer set inn
106,palencia,ES,42.4167,-4.5000,30,Hotel Área de Servicio Los Chopos
107,inhambane,MZ,-23.8650,35.3833,65,Hotel de Inhambane


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

import hvplot.pandas

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",  # Longitude for the x-axis
    "Lat",  # Latitude for the y-axis
    geo=True,  # Enable geographic plotting
    tiles='OSM',  # Use OpenStreetMap tiles
    size=10,  # Set a fixed size for the points
    color="City",  # Set the color based on the city names
    frame_width=800,  # Set the width of the map
    frame_height=600,  # Set the height of the map
    hover_cols=['City', 'Country', 'Hotel Name'],  # Add hotel name and country in hover message
    legend=True  # Enable legend to identify cities by color
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)